<a href="https://colab.research.google.com/github/svantepihl/Thesis-MaskDetection/blob/master/train_model_k_fold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`NOTE!`  
`This notebook needs to be ran in a TPU environment`

`Dataset loads from a private GCP bucket but can be downloaded from here :` 

[LINK](https://drive.google.com/drive/folders/18UJsRrjrW4lIlKbYhNQbQcrLFoApxymp?usp=sharing)

# Imports

In [ ]:
from datetime import datetime
import math
import re
import os
import time
import sys
import numpy as np
import pandas as pd
import sklearn
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Installs

In [ ]:
!pip install -U tensorboard_plugin_profile # Tensorboard plugin for profiling performance 

# Auth for Google Cloud Plattform.

In [ ]:
if 'google.colab' in sys.modules:
   from google.colab import auth
   auth.authenticate_user()

# Tensorflow version

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf, tensorflow.keras.backend as K
print("Tensorflow version " + tf.__version__)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

# TPU config  
Reference: [TPUs in Google Colab](https://colab.research.google.com/notebooks/tpu.ipynb)

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

# Constants

All constants are captitalized

In [ ]:
#@title Default title text

SEED = 1337

AUTO = tf.data.experimental.AUTOTUNE

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_CHANNELS = 3
IMAGE_SIZE = [IMAGE_WIDTH, IMAGE_HEIGHT]
IMAGE_SHAPE = [*IMAGE_SIZE, IMAGE_CHANNELS]

EPOCHS = 150

# TRANINGS SETTINGS
if tpu != None:
  BATCH_SIZE = 128 * strategy.num_replicas_in_sync
  VALIDATION_BATCH_SIZE = 128 * strategy.num_replicas_in_sync
  TEST_BATCH_SIZE = 128 * strategy.num_replicas_in_sync
  AUG_BATCH = BATCH_SIZE
else:
  BATCH_SIZE = 64
  VALIDATION_BATCH_SIZE = 64
  TEST_BATCH_SIZE = 64
  AUG_BATCH = BATCH_SIZE

GCS_DATASET_PATTERN = 'gs://facemask-detection-thesis-32-tfrecords-jpeg-224x224/*.tfrec' # GCS bucket where dataset is stored

GCS_LOG_BUCKET = 'gs://facemask-detection-thesis-training-logs/' # To store training logs for tensorboard

NEW_RUN = False #@param {type:"boolean"}
if NEW_RUN:
  now = datetime.now()
  dt_string = now.strftime("%Y-%m-%d_%H")

  RUN_FOLDER = '/content/drive/MyDrive/MaskedFace/K-Folds/RUN-' + dt_string +'/'
  os.makedirs(RUN_FOLDER)
  print("Created folder: "+ RUN_FOLDER)

  MODEL_FOLDER = RUN_FOLDER + 'Models/'
  os.makedirs(MODEL_FOLDER)
  print("Created folder: "+ MODEL_FOLDER)

CLASSES = ['MaskCorrect', 'MaskOnChin', 'MaskOnlyOnMouth', 'NoMask']

TEST_TRAIN_SPLIT = 0.15

In [ ]:
print("Previous runs:")
print(os.listdir("/content/drive/MyDrive/MaskedFace/K-Folds/"))

In [ ]:
#@title Select previous run
PREVIOUS_RUN = "RUN-2021-05-05_09" #@param {type:"string"}
PREVIOUS_RUN_FOLDER = '/content/drive/MyDrive/MaskedFace/K-Folds/' + PREVIOUS_RUN + '/'

PREVIOUS_MODELS_FOLDER = PREVIOUS_RUN_FOLDER + '/Models/'

print('PREVIOUS_RUN_FOLDER: '+ PREVIOUS_RUN_FOLDER)
print('PREVIOUS_MODELS_FOLDER: '+ PREVIOUS_MODELS_FOLDER)

# Utility functions

In [ ]:
def write_vars_to_file(f, **kwargs):
    for name, val in kwargs.items():
      f.write("%s = %s\n" %(name, val))

def get_dataset_labels(dataset):
  _, labels = tuple(zip(*dataset.unbatch()))
  labels = np.array(labels)
  return labels

def dataset_to_numpy_util(dataset,N):
  dataset = dataset.unbatch().batch(N)
  for images, labels in dataset:
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    break;  
  return numpy_images, numpy_labels

def whole_dataset_to_numpy_util(dataset):
  images, labels = tuple(zip(*dataset.unbatch()))
  images = np.array(images)
  labels = np.array(labels)
  return images,labels

def title_from_label_and_target(label, correct_label):
  label = np.argmax(label, axis=-1)  # one-hot to class number
  correct_label = np.argmax(correct_label, axis=-1) # one-hot to class number
  correct = (label == correct_label)
  return "{} [{}{}{}]".format(CLASSES[label], str(correct), ', shoud be ' if not correct else '',
                              CLASSES[correct_label] if not correct else ''), correct

def display_one_grayscale_image(image, title, subplot, red=False):
    plt.subplot(subplot)
    plt.axis('off')
    arr = np.asarray(image)
    arr = arr[:,:,0]
    plt.imshow(arr, cmap='gray', vmin=0, vmax=255) # Grayscale 
    plt.title(title, fontsize=16, color='red' if red else 'black')
    return subplot+1

def display_one_image(image, title, subplot, red=False):
    plt.subplot(subplot)
    plt.axis('off')
    plt.imshow(image, cmap='gray', vmin=0, vmax=255) # color
    plt.title(title, fontsize=16, color='red' if red else 'black')
    return subplot+1
  
def display_9_images_from_dataset(dataset, grayscale = False):
  subplot=331
  plt.figure(figsize=(13,13))
  images, labels = dataset_to_numpy_util(dataset, 9)
  for i, image in enumerate(images):
    title = CLASSES[np.argmax(labels[i], axis=-1)]
    if grayscale:
      subplot = display_one_grayscale_image(image, title, subplot)
    else:
      subplot = display_one_image(image, title, subplot)
    if i >= 8:
      break;
              
  plt.tight_layout() 
  plt.subplots_adjust(wspace=0.1, hspace=0.1)
  plt.show()
  
def display_9_images_with_predictions(images, predictions, labels):
  subplot=331
  plt.figure(figsize=(13,13))
  for i, image in enumerate(images):
    title, correct = title_from_label_and_target(predictions[i], labels[i])
    subplot = display_one_image(image, title, subplot, not correct)
    if i >= 8:
      break;
              
  plt.tight_layout() 
  plt.subplots_adjust(wspace=0.1, hspace=0.1)
  plt.show()
  
def display_training_curves(training, validation, title, subplot):
  if subplot%10==1: # set up the subplots on the first call
    plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
    plt.tight_layout()
  ax = plt.subplot(subplot)
  ax.set_facecolor('#F8F8F8')
  ax.plot(training)
  ax.plot(validation)
  ax.set_title('model '+ title)
  ax.set_ylabel(title)
  #ax.set_ylim(0.28,1.05)
  ax.set_xlabel('epoch')
  ax.legend(['train', 'valid.'])

# Augmentations

## Rotate, shear, shift, zoom

In [ ]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))

In [ ]:
def rotate_shear_shift_zoom(image,label):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SIZE[0]
    XDIM = DIM%2 #fix for size 331
    
    rot = 14. * tf.random.normal([1],dtype='float32')
    shr = 5. * tf.random.normal([1],dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    w_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    h_shift = 16. * tf.random.normal([1],dtype='float32') 
    w_shift = 16. * tf.random.normal([1],dtype='float32') 
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    

    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
    
    image = tf.reshape(d,[DIM,DIM,3])
    return image,label

## Image augmentations

In [ ]:
def img_augment(image, one_hot_class):
    image = tf.image.random_hue(image, 0.08,seed=SEED) 
    image = tf.image.random_saturation(image, 0.6, 1.6,seed=SEED)
    image = tf.image.random_contrast(image, 0.7, 1.3,seed=SEED)
    image = tf.image.random_flip_left_right(image,seed=SEED)
    image = tf.image.random_brightness(image,0.5)
    return image, one_hot_class

# Dataset functions

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    # image = (tf.cast(image, tf.float32) / 127.5) - 1  # convertion done in model
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

In [ ]:
def read_tfrecord(example):
  features = {
    "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
    "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means scalar
    "one_hot_class": tf.io.VarLenFeature(tf.float32),
  }
  example = tf.io.parse_single_example(example, features)
  image = decode_image(example['image'])
  one_hot_class = tf.reshape(tf.sparse.to_dense(example['one_hot_class']), [4])
  label = tf.cast(example['class'], tf.int32)
  return image, one_hot_class

In [ ]:
def load_dataset(filenames, ordered = False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # Diregarding data order. Order does not matter since we will be shuffling the data anyway
    
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
        
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # use data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_tfrecord, num_parallel_calls = AUTO) # returns a dataset of (image, label) pairs if labeled = True or (image, id) pair if labeld = False
    return dataset

In [ ]:
def get_training_dataset(dataset):
    dataset = dataset.repeat()
    dataset = dataset.map(img_augment, num_parallel_calls=AUTO)
    dataset = dataset.map(rotate_shear_shift_zoom, num_parallel_calls=AUTO)
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

In [ ]:
def get_validation_dataset(dataset):
    dataset = dataset.batch(VALIDATION_BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

In [ ]:
def get_test_dataset(ordered=True,prefetch=True,batched=True):
    dataset = load_dataset(TEST_FILENAMES, ordered=ordered)
    if batched:
      dataset = dataset.batch(BATCH_SIZE)
    if prefetch: 
      dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

# Load data from GCS

In [ ]:
%%time
filenames = tf.io.gfile.glob(GCS_DATASET_PATTERN)

# Split into train/test

In [ ]:
TRAIN_FILENAMES, TEST_FILENAMES = sklearn.model_selection.train_test_split(filenames, test_size=TEST_TRAIN_SPLIT)

# Summarise dataset

In [ ]:
def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. 00-2000.tfrec = 2000 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [ ]:
NUM_TRAINING_IMAGES = count_data_items(TRAIN_FILENAMES) * 0.80
NUM_VALIDATION_IMAGES = count_data_items(TRAIN_FILENAMES) * 0.20
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

TRAIN_STEPS = count_data_items(TRAIN_FILENAMES) // BATCH_SIZE
print("TOTAL IMAGES: ", int(count_data_items(filenames)))
print("TRAINING IMAGES: ", int(NUM_TRAINING_IMAGES), ", STEPS PER EPOCH: ", int(STEPS_PER_EPOCH))
print("VALIDATION IMAGES ",int(NUM_VALIDATION_IMAGES))
print("TESTING IMAGES: ", int(NUM_TEST_IMAGES))

# Callbacks

## Learning rate

In [ ]:
# Learning rate settings
start_lr = 0.0000001 
min_lr = 0.00000001
if tpu != None:
  max_lr = 0.00005 * strategy.num_replicas_in_sync
else: 
  max_lr = 0.00005
rampup_epochs = 50
sustain_epochs = 10
exp_decay = .6

# Learning rate function
def learning_rate_fn(epoch):
    def lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay):
        if epoch < rampup_epochs:
            lr = (max_lr - start_lr)/rampup_epochs * epoch + start_lr
        elif epoch < rampup_epochs + sustain_epochs:
            lr = max_lr
        else:
            lr = (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr
        return lr
    return lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay)
    


# Plot learning rate
rng = [i for i in range(EPOCHS)]
y = [learning_rate_fn(x) for x in rng]
plt.plot(rng, [learning_rate_fn(x) for x in rng])
print(y[0], y[-1])

### Save learning rate settings 

In [ ]:
lr_file = open(RUN_FOLDER + 'LR.txt',mode='a+')
write_vars_to_file(lr_file,
                   start_lr=start_lr,
                   min_lr=min_lr,
                   max_lr=max_lr,
                   rampup_epochs=rampup_epochs,
                   sustain_epochs=sustain_epochs,
                   exp_decay=exp_decay)
lr_file.close()


In [ ]:
def get_learning_rate():
  return tf.keras.callbacks.LearningRateScheduler(lambda epoch: learning_rate_fn(epoch), verbose=True)

## Early stopping

In [ ]:
def get_earlystopping_callback(epoch_patience = 10):
  return tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=epoch_patience,
    verbose=True
  )

## Model checkpoint

In [ ]:
def get_checkpoint_callback(model_name):
  checkpoint_path = MODEL_FOLDER + model_name
  return tf.keras.callbacks.ModelCheckpoint(checkpoint_path,monitor='val_loss',save_best_only=True)

## Tensorboard callback

In [ ]:
def get_tensorboard_callback():
  logs = GCS_LOG_BUCKET + datetime.now().strftime("%Y%m%d-%H%M%S")
  return tf.keras.callbacks.TensorBoard(log_dir = logs, 
                                        histogram_freq = 1,
                                        profile_batch=(20,50))

# Load and start Tensorboard

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
# Get TPU profiling service address. This address will be needed for capturing
# profile information with TensorBoard in the following steps.
service_addr = tpu.get_master().replace(':8470', ':8466')
tpu_worker = os.environ['COLAB_TPU_ADDR'].replace('8470', '8466') 
print(tf.profiler.experimental.client.monitor(tpu_worker,1))
print(tpu_worker)

In [ ]:
# Launch TensorBoard.
%tensorboard --logdir=gs://facemask-detection-thesis-training-logs/  

# Create model

In [ ]:
def create_model():
  # Base model
  base_model = tf.keras.applications.MobileNetV3Small(
    input_shape=IMAGE_SHAPE,
    minimalistic=True, 
    include_top=False,
    weights='imagenet'
  )

  input = tf.keras.Input(shape=IMAGE_SHAPE)
  x = tf.keras.applications.mobilenet_v3.preprocess_input(input)
  x = base_model(x)
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  x = tf.keras.layers.Dense(128,activation='relu')(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  outputs = tf.keras.layers.Dense(4, activation='softmax')(x)

  model = tf.keras.Model(input, outputs)

  model.compile(
      optimizer='adam',
      loss = 'categorical_crossentropy',
      metrics=['categorical_accuracy']
  )
  return model

In [ ]:
def create_tpu_model():
  with strategy.scope():
    return create_model()

## Save model settings

In [ ]:
model = create_model()
model.summary()
json_model = model.to_json()
with open(RUN_FOLDER+'model.json', 'w') as json_file:
    json_file.write(json_model)
del model, json_model

# Training

In [ ]:
def train_cross_validate(folds = 5):
    histories = []
    models = []
    early_stopping = get_earlystopping_callback()
    tensorboard = get_tensorboard_callback()
    learning_rate = get_learning_rate()
    kfold = sklearn.model_selection.KFold(folds, shuffle = True, random_state = SEED)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(TRAIN_FILENAMES)):
        print(); print('#'*25)
        print('### FOLD',fold+1)
        print('#'*25)
        train_dataset = load_dataset(list(pd.DataFrame({'TRAINING_FILENAMES': TRAIN_FILENAMES}).loc[trn_ind]['TRAINING_FILENAMES']), ordered = False)
        val_dataset = load_dataset(list(pd.DataFrame({'TRAINING_FILENAMES': TRAIN_FILENAMES}).loc[val_ind]['TRAINING_FILENAMES']),  ordered = True)

        checkpoint_name = f'model_fold_{fold + 1}' + '.h5'
        model_checkpoint = get_checkpoint_callback(checkpoint_name)
        
        model = create_tpu_model()

        history = model.fit(
                    get_training_dataset(train_dataset), 
                    validation_data=get_validation_dataset(val_dataset),
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    epochs=EPOCHS, 
                    callbacks=[early_stopping,tensorboard,learning_rate,model_checkpoint])
        
        print('Load best weights for model prediction')
        model.load_weights(MODEL_FOLDER + checkpoint_name)
        models.append(model)
        histories.append(history)
        
    return histories, models

In [ ]:
def train_and_test(folds = 5):
    print("Loading test data")
    test_dataset = get_test_dataset(ordered=True)
    test_labels = get_dataset_labels(test_dataset)
    print('Start training %i folds'%folds)
    histories, models = train_cross_validate(folds = folds)
    print('Computing predictions...')
    # get the mean probability of the folds models
    probabilities = np.average([models[i].predict(test_dataset,verbose=1) for i in range(folds)], axis = 0)
    predictions = np.argmax(probabilities, axis=-1)
    label_true = np.argmax(test_labels, axis=-1)
    print('Printing classification report...')
    print(classification_report(label_true, predictions, target_names=CLASSES))
    
    return histories, models

In [ ]:
histories, models = train_and_test()

# Evaluate models

## Load dataset

In [ ]:
print("Loading test labels..")
test_dataset = get_test_dataset(ordered=True)
test_labels = get_dataset_labels(test_dataset)
print("Test data loaded")

## Run evaluation

In [ ]:

evaluations = []
predictions = []

for fold,model in enumerate(models):
  print("=================== Test av modell från fold " + str(fold+1) + " ===================\n")
  evaluation = model.evaluate(test_dataset,verbose=0)
  evaluations.append(evaluation)
  probabilities = model.predict(test_dataset,verbose=0)
  prediction = np.argmax(probabilities, axis=-1)
  predictions.append(prediction)
  label_true = np.argmax(test_labels, axis=-1)
  print(classification_report(label_true, prediction, target_names=CLASSES,digits=6))
  print('      test loss  ', round(evaluations[0][0],6))
  print('\n')
  time.sleep(5)

  
  